In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [2]:
df = pd.read_csv("/kaggle/input/daphnet-windows/daphnet_windows.csv")
df

,input0,input1,input2,input3,input4,input5,input6,input7,input8,input9,...,input759,input760,input761,input762,input763,input764,input765,input766,input767,label
0,-45.0,-18.0,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,...,181.0,202.0,202.0,171.0,202.0,212.0,202.0,202.0,191.0,0
1,-18.0,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,...,202.0,202.0,171.0,202.0,212.0,202.0,202.0,191.0,181.0,0
2,18.0,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,...,202.0,171.0,202.0,212.0,202.0,202.0,191.0,181.0,222.0,0
3,36.0,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,-163.0,...,171.0,202.0,212.0,202.0,202.0,191.0,181.0,222.0,212.0,0
4,36.0,27.0,27.0,-18.0,-90.0,-90.0,-136.0,-145.0,-163.0,-236.0,...,202.0,212.0,202.0,202.0,191.0,181.0,222.0,212.0,202.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094283,-109.0,-109.0,-109.0,-109.0,-109.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1094284,-109.0,-109.0,-109.0,-109.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1094285,-109.0,-109.0,-109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1094286,-109.0,-109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
SR = 64            # Sample rate in hertz
stepSize = 1       # Step size in samples
offDelay = 2       # Evaluation delay in seconds: tolerates delay after detecting
onDelay = 2        # Evaluation delay in seconds: tolerates delay before detecting

NFFT = windowLength = 256
locoBand = np.array([0.5, 3])
freezeBand = np.array([3, 8])

f_res = SR / NFFT
f_nr_LBs = int(np.round(locoBand[0] / f_res))
f_nr_LBe = int(np.round(locoBand[1] / f_res))
f_nr_FBs = int(np.round(freezeBand[0] / f_res))
f_nr_FBe = int(np.round(freezeBand[1] / f_res))

d = NFFT/2

freezeTH = 1.5
powerTH = 2 ** 11.5

def find(X, n=None, direction=None):
    if direction != None and n != None:
        if direction == "last": return np.where(X)[-n:]
        else: return np.where(X)[:n]
    return np.where(X)


def numIntegration(x):
    """
    Do numerical integration of x
    """
    return (np.sum(x[1:])/SR+np.sum(x[:-1])/SR)/2

In [4]:
def fi(signal): # Moore's Algorithm
    """
    Compute the freezing index
    """
    
    signal -= np.mean(signal) # make signal zero-mean (mean normalization)
    
    # compute FFT (Fast Fourier Transform)
    Y = sp.fft.fft(signal, NFFT)
    Pyy = Y * np.conj(Y) / NFFT
    
    # --- calculate sumLocoFreeze and freezeIndex ---
    areaLocoBand = numIntegration(Pyy[f_nr_LBs:f_nr_LBe])
    areaFreezeBand = numIntegration(Pyy[f_nr_FBs:f_nr_FBe])
    
    # Extension of Baechlin to handle low-energy situations (e.g. standing)
    return areaFreezeBand / areaLocoBand if areaFreezeBand + areaLocoBand >= powerTH else 0

def fIndex(row):
    return pd.Series(dict(
        freezeX = fi(row.iloc[:256].values),
        freezeY = fi(row.iloc[256:512].values),
        freezeZ = fi(row.iloc[512:768].values),
        label = row["label"]
    ))
    

In [9]:
freezes = df.apply(fIndex, axis=1).astype(float)
freezes.label = freezes.label.astype(int)
freezes

,freezeX,freezeY,freezeZ,label
0,11.901779,0.0,0.000000,0
1,11.862690,0.0,0.000000,0
2,11.965667,0.0,0.000000,0
3,12.110931,0.0,0.000000,0
4,12.198498,0.0,0.000000,0
...,...,...,...,...
1094283,0.000000,0.0,1.233355,0
1094284,0.000000,0.0,1.507971,0
1094285,0.000000,0.0,1.764333,0
1094286,0.000000,0.0,1.973801,0


In [21]:
freezes_fixed = freezes.drop_duplicates(ignore_index=False)
freezes_fixed

,freezeX,freezeY,freezeZ,label
0,11.901779,0.0,0.000000,0
1,11.862690,0.0,0.000000,0
2,11.965667,0.0,0.000000,0
3,12.110931,0.0,0.000000,0
4,12.198498,0.0,0.000000,0
...,...,...,...,...
1094280,0.000000,0.0,0.539964,0
1094282,0.000000,0.0,0.967602,0
1094283,0.000000,0.0,1.233355,0
1094284,0.000000,0.0,1.507971,0


In [14]:
freezes_fixed.describe()

,freezeX,freezeY,freezeZ,label
count,688992.000000,688992.000000,688992.000000,688992.000000
mean,2.618950,2.253492,2.974321,0.156145
std,6.929811,6.159725,6.333354,0.362993
min,0.000000,0.000000,0.000000,0.000000
25%,0.765288,0.292194,0.780070,0.000000
50%,1.086700,0.631801,1.371616,0.000000
75%,1.829430,1.761402,2.575389,0.000000
max,214.178587,174.481673,211.663876,1.000000


In [20]:
freezes_fixed.label.value_counts()

0    581409
1    107583
Name: label, dtype: int64

In [22]:
freezes_fixed.to_csv("findex.csv", index=False)

In [29]:
X = freezes_fixed.iloc[:, :3].values
y = freezes_fixed.label.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)